[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1p1nL8leOeuug2la1nlEeYcCxUG0dApXh?usp=sharing)

[説明資料](https://drive.google.com/file/d/1slSX1N3bSKhEhOg-z3dJEMXTYv5fNEFp/view?usp=drive_link)  
※ RetrievalQA -> p.104～

### **Config**

---



In [ ]:
import torch
class Config:
  PROJECT_FOLD_NAME = 'CDLE_LLM'

  # LLM
  ## OpenAI
  GPT_MAX_TOKENS = 256
  GPT_MODEL_NAME = 'gpt-3.5-turbo'

  # その他
  is_default_verbose    = True
  device                = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'


class QAConfig:
  # RetrievalQA 用
  ## EMBEDDING
  ## https://huggingface.co/models?pipeline_tag=sentence-similarity&language=ja&sort=likes
  EMBED_MODEL_NAME_OR_PATH = 'intfloat/multilingual-e5-large'

  ## チャンクサイズの設定
  splitter_chunk_size    = 514 # 指定されるチャンクサイズ
  splitter_chunk_overlap = 20  # 指定されるチャンクサイズのバッファ

  ## Index の保存
  IS_SAVE_INDEX = True

  ## QA回答の設定
  chain_type = 'stuff' # 回答の生成方法

class TOKEN:
  OPENAI_API_KEY = '*** YOUR OPEN AI KEY ***'
  # MEMO
  # openai:        https://platform.openai.com/account/api-keys

In [ ]:
cfg    = Config()
qa_cfg = QAConfig()
auth   = TOKEN()

### **Install Library**

---



> #### pip

In [ ]:
from IPython.display import clear_output
# langchain
!pip install langchain==0.0.271

# HuggingFace transformers
!pip install transformers==4.31.0 xformers accelerate==0.22.0

# OpenAI
!pip install openai

## RetrievalQA 用
!pip install sentence_transformers
!pip install faiss-gpu
### USE PyPDFLoader
!pip install pypdf

clear_output()

> #### import Library

In [ ]:
import os
import sys
import glob
import random

import numpy as np
import pandas as pd
pd.set_option('display.max_rows',     50)
pd.set_option('display.max_columns',  100)
pd.set_option('display.max_colwidth', 100)
from typing import Dict, List, Union, Optional, Type

import re
import time
import pytz
from datetime import datetime

from tqdm.notebook import tqdm
# TQDM Progress Bar With Pandas Apply Function
tqdm.pandas()
from contextlib import contextmanager
from IPython.display import clear_output
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect();

In [ ]:
# langchain
## 共通
import langchain
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

## RetrievalQA 用
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
## Chatのストーリーミング出力用(OpenAI)
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# HuggingFace transformers
import transformers

# torch
import torch

# Colabo Bug?
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

print(f'Python Version: {sys.version}')
print(f'langchain Version: {langchain.__version__}')
print(f'transformers Version: {transformers.__version__}')
print(f'torch Version: {torch.__version__}')

# Debugging chains
langchain.debug = cfg.is_default_verbose

Python Version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
langchain Version: 0.0.271
transformers Version: 4.31.0
torch Version: 2.1.0+cu118


> #### API Keys set

In [ ]:
os.environ['OPENAI_API_KEY'] = auth.OPENAI_API_KEY if auth.OPENAI_API_KEY else 'none'

### **ディレクトリの作成**

---


In [ ]:
def clear_cache_everything():
    gc.collect();
    try: torch.cuda.empty_cache();
    except: pass;

In [ ]:
if not os.path.exists(f'{cfg.PROJECT_FOLD_NAME}'):
  !mkdir {cfg.PROJECT_FOLD_NAME}

## RetrievalQA 用
if not os.path.exists(f'{cfg.PROJECT_FOLD_NAME}/qa_documents'):
  !mkdir {cfg.PROJECT_FOLD_NAME}/qa_documents
if not os.path.exists(f'{cfg.PROJECT_FOLD_NAME}/qa_documents/txt'):
  !mkdir {cfg.PROJECT_FOLD_NAME}/qa_documents/txt
if not os.path.exists(f'{cfg.PROJECT_FOLD_NAME}/qa_documents/pdf'):
  !mkdir {cfg.PROJECT_FOLD_NAME}/qa_documents/pdf
if not os.path.exists(f'{cfg.PROJECT_FOLD_NAME}/qa_index'):
  !mkdir {cfg.PROJECT_FOLD_NAME}/qa_index

### **RetrievalQA**

---



https://python.langchain.com/docs/use_cases/question_answering/

> #### **文章のインデックス化(FAISS)**

> > ##### func CreateIndex

In [ ]:
def files2documents(cfg, qa_cfg):
  clear_cache_everything();

  # txtとpdfを読み取り
  load_file_type   = ['txt', 'pdf']
  return_documents = []
  files_list       = []

  # チャンクの分割の設定
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size    = qa_cfg.splitter_chunk_size,
      chunk_overlap = qa_cfg.splitter_chunk_overlap,)

  for f_type in load_file_type:
    tmp_l      = [f'{cfg.PROJECT_FOLD_NAME}/qa_documents/{f_type}/{f}' for f in os.listdir(f'{cfg.PROJECT_FOLD_NAME}/qa_documents/{f_type}')]
    files_list += tmp_l

  if not len(files_list) == 0:
    for f in files_list:
      if f.endswith('.txt'):   raw_documents = TextLoader(f, encoding='utf8').load()
      elif f.endswith('.pdf'): raw_documents = PyPDFLoader(f).load()
      else: print(f'WARNING: file format error: {f}'); continue;

      # チャンクの分割
      documents        = text_splitter.split_documents(raw_documents)
      return_documents += documents

    clear_cache_everything();
    return return_documents

  else: print('file not found...?'); return None;

In [ ]:
# https://python.langchain.com/docs/modules/data_connection/vectorstores/
# https://python.langchain.com/docs/integrations/vectorstores/faiss
def CreateIndex(cfg, qa_cfg, documents):
  clear_cache_everything();

  huggingface_embeddings = HuggingFaceEmbeddings(model_name = qa_cfg.EMBED_MODEL_NAME_OR_PATH)
  clear_output();

  index = FAISS.from_documents(
    documents = documents,
    embedding = huggingface_embeddings,)

  # 作成した index の保存
  if qa_cfg.IS_SAVE_INDEX:
    index.save_local(f'{cfg.PROJECT_FOLD_NAME}/qa_index')

  clear_cache_everything();
  return index

def LoadIndex(cfg, qa_cfg):
  clear_cache_everything();

  huggingface_embeddings = HuggingFaceEmbeddings(model_name = qa_cfg.EMBED_MODEL_NAME_OR_PATH)
  clear_output();

  index = FAISS.load_local(f'{cfg.PROJECT_FOLD_NAME}/qa_index', huggingface_embeddings)

  clear_cache_everything();
  return index

> > ##### インデックス化の実行

In [ ]:
# チャンクの分割
documents = files2documents(cfg, qa_cfg)
# インデックスの新規作成
index     = CreateIndex(cfg, qa_cfg, documents)
# インデックスの読み込み
# index     = LoadIndex(cfg, qa_cfg)

In [ ]:
# チャンクの確認
print(len(documents))
for document in documents[:5]:
    print(document.page_content[:15].replace('\n','\\n'),len(document.page_content), document.metadata)

98
名探偵コナン\n\nテレビアニメを 321 {'source': 'CDLE_LLM/qa_documents/txt/conan.txt'}
巻数\n既刊103巻\n\n世界観\n 352 {'source': 'CDLE_LLM/qa_documents/txt/conan.txt'}
また、1994年の連載開始から 260 {'source': 'CDLE_LLM/qa_documents/txt/conan.txt'}
作品構成\n主に1エピソードにつ 303 {'source': 'CDLE_LLM/qa_documents/txt/conan.txt'}
事件を推理し解決に導くのはコナ 386 {'source': 'CDLE_LLM/qa_documents/txt/conan.txt'}


In [ ]:
# indexでsimilarity_search確認
res = index.similarity_search('ギターヒーロー', k=3)
contents = [f'doc {i+1}:\n {res[i].page_content}' for i in range(len(res))]
join_contents = '\n\n'.join(contents)
print(join_contents)

doc 1:
 ギターヒーロー
後藤ひとりが動画配信の際に用いるハンドルネーム。ひとりは「ギターヒーロー」の名で動画配信して、スゴ腕の女子高生ギタリストとしてカリスマ的な人気を集めている。動画配信は素顔を映さない形で行っており、ひとりがギターヒーローであることはほとんどの人たちが知らない。チャンネル登録者もかなり多いらしく、大槻ヨヨコによれば、ドームを2回満員にするほどにチャンネル登録者がいるという。ただ一方で、ひとりの承認欲求が肥大化した原因でもあり、ギターヒーローとしての活動コメントは、「彼氏がいる」「友達とカラオケ」など捏造がひどい。実はアカウントが家族で共有されているため、ひとりの父にはそれらの虚言癖は最初からバレており、のちにその事実に気づく。また佐藤愛子にひとりがギターヒーローであることを暴露されたことをきっかけに周囲の人間にもそれが知れ渡り、以降は激しく後悔してコメントを自粛している。

doc 2:
 実力派サイケデリックロックバンド「SICK

doc 3:
 文化祭ライブ


> #### **RetrievalQA**

> > ##### Define Prompt

**RetrievalQA用**

In [ ]:
qa_prompt_template = """Please answer in Japanese. Please make your answer as concise as possible. Answer the following questions as best you can. Answer the Question by referring only to the Context below. If the Context does not contain information that will help answer the question, please say "I don't know". If there are multiple possible answers, please list them.

Context:.
{context}

Begin!

Question:{question}
Answer:"""

In [ ]:
DEFINE_PROMPT = dict()

# RetrievalQA
DEFINE_PROMPT.update({
    'qa_prompt_template': qa_prompt_template,
})

> > ##### func CreateRetrievalQA

In [ ]:
def CreateRetrievalQA(
          cfg, qa_cfg, index_, DEFINE_PROMPT, *,
          ## Agent LLM
          temperature:float            = 0.6,
          top_p:float                  = 0.95,
          # Streamer
          IS_STREMING:bool             = True,
          # RetrievalQA
          retriever_topk:int           = 5,
          return_source_documents:bool = True,
          verbose:bool                 = False,):
  clear_cache_everything();

  # Create LLM
  llm = OpenAI(
          model_name           = cfg.GPT_MODEL_NAME,
          streaming            = True if IS_STREMING else False,
          max_tokens           = cfg.GPT_MAX_TOKENS,
          temperature          = temperature,
          top_p                = top_p,
          callback_manager     = CallbackManager([StreamingStdOutCallbackHandler()]),)

  # Prompt
  QA_PROMPT_TEMPLATE = PromptTemplate(
          input_variables = ['context', 'question'],
          template        = DEFINE_PROMPT['qa_prompt_template'],)

  # Create Chain
  qa_chain = RetrievalQA.from_chain_type(
          llm                     = llm,
          chain_type              = qa_cfg.chain_type,
          retriever               = index_.as_retriever(k = retriever_topk),
          chain_type_kwargs       = {'prompt': QA_PROMPT_TEMPLATE},
          return_source_documents = return_source_documents,
          verbose                 = verbose,)
  # MEMO:
  # retriever_topk:          呼び出されるチャンクの量
  # return_source_documents: 参照したリソースの表示
  # chain_type:
  #   stuff:      すべての chunk を含む prompt を実行する。一度に全ての chunk を詰め込むため、大きなデータは使えない。
  #   map_reduce: retriever の chunk ごとに prompt を実行して、最終的にそれを結合。情報の結合時に情報が失われる。
  #   refine:     最初の chunk に prompt を実行した回答に次の chunk を含めて再度 prompt の実行を繰り返す。
  #   map_rerank: retriever の chunk ごとに prompt を実行し確からしさのスコアリングを行う。最高スコアを回答する。chunk 間の情報は失われる。
  #   https://python.langchain.com/docs/modules/chains/document/

  clear_cache_everything();
  return qa_chain, llm

> > ##### RetrievalQAの作成と実行

**Helper**

In [ ]:
def pick_metadata_source(result:dict, sort_reverse:bool = False) -> list:
  source_list=[]
  for i in range(len(result['source_documents'])):
    source_list.append(result['source_documents'][i].metadata['source'])
  return sorted(list(set(source_list)), reverse=sort_reverse)

**RetrievalQAの作成**

In [ ]:
qa_chain, original_llm = CreateRetrievalQA(cfg, qa_cfg, index, DEFINE_PROMPT)

**実行**

In [ ]:
query = 'ギターヒーローは誰ですか？'
generate = original_llm(query)
print('回答:', generate)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "ギターヒーローは誰ですか？"
  ]
}
ギターヒーローは、ゲーム『Guitar Hero』シリーズのプレイヤーキャラクターを指すことが一般的です。ゲーム内では、プレイヤー自身がギターを演奏する「ヒーロー」として扱われます。具体的なキャラクター名はありませんが、プレイヤーが自分自身を表現することができるため、プレイヤーごとに異なる「ギターヒーロー」として楽しむことができます。[llm/end] [1:llm:OpenAIChat] [13.64s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "ギターヒーローは、ゲーム『Guitar Hero』シリーズのプレイヤーキャラクターを指すことが一般的です。ゲーム内では、プレイヤー自身がギターを演奏する「ヒーロー」として扱われます。具体的なキャラクター名はありませんが、プレイヤーが自分自身を表現することができるため、プレイヤーごとに異なる「ギターヒーロー」として楽しむことができます。",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
回答: ギターヒーローは、ゲーム『Guitar Hero』シリーズのプレイヤーキャラクターを指すことが一般的です。ゲーム内では、プレイヤー自身がギターを演奏する「ヒーロー」として扱われます。具体的なキャラクター名はありませんが、プレイヤーが自分自身を表現することができるため、プレイヤーごとに異なる「ギターヒーロー」として楽しむことができます。


In [ ]:
query = 'ギターヒーローは誰ですか？'
generate = qa_chain(query)
print('*'*10, '\n回答:',    generate['result'])
print('*'*10, '\nSource: ', pick_metadata_source(generate))

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "ギターヒーローは誰ですか？"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "ギターヒーローは誰ですか？",
  "context": "ギターヒーロー\n後藤ひとりが動画配信の際に用いるハンドルネーム。ひとりは「ギターヒーロー」の名で動画配信して、スゴ腕の女子高生ギタリストとしてカリスマ的な人気を集めている。動画配信は素顔を映さない形で行っており、ひとりがギターヒーローであることはほとんどの人たちが知らない。チャンネル登録者もかなり多いらしく、大槻ヨヨコによれば、ドームを2回満員にするほどにチャンネル登録者がいるという。ただ一方で、ひとりの承認欲求が肥大化した原因でもあり、ギターヒーローとしての活動コメントは、「彼氏がいる」「友達とカラオケ」など捏造がひどい。実はアカウントが家族で共有されているため、ひとりの父にはそれらの虚言癖は最初からバレており、のちにその事実に気づく。また佐藤愛子にひとりがギターヒーローであることを暴露されたことをきっかけに周囲の人間にもそれが知れ渡り、以降は激しく後悔してコメントを自粛している。\n\n秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に苦手としている。そのため、すぐに自分の世界に入って落ち込むという情緒不安定さを見せる。押し入れやダンボールに潜り込む癖があり、「完熟マンゴー仮面」を持ち歩いている。運動も勉強も苦手で、特に勉強はまじめに授業を受けているにもかかわらず、生来の要領の悪さから赤点ギリギリ。

In [ ]:
query = 'コナンはなぜ子どもの姿になったのですか？'
generate = original_llm(query)
print('回答:', generate)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "コナンはなぜ子どもの姿になったのですか？"
  ]
}
コナンは、組織と呼ばれる秘密結社によって毒薬を使われてしまい、成人の姿から子どもの姿に変えられてしまいました。この変身は、彼を殺すために行われたものであり、組織のメンバーに見つかることを避けるために、彼は子どもの姿で生活するようになりました。[llm/end] [1:llm:OpenAIChat] [10.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "コナンは、組織と呼ばれる秘密結社によって毒薬を使われてしまい、成人の姿から子どもの姿に変えられてしまいました。この変身は、彼を殺すために行われたものであり、組織のメンバーに見つかることを避けるために、彼は子どもの姿で生活するようになりました。",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
回答: コナンは、組織と呼ばれる秘密結社によって毒薬を使われてしまい、成人の姿から子どもの姿に変えられてしまいました。この変身は、彼を殺すために行われたものであり、組織のメンバーに見つかることを避けるために、彼は子どもの姿で生活するようになりました。


In [ ]:
query = 'コナンはなぜ子どもの姿になったのですか？'
generate = qa_chain(query)
print('*'*10, '\n回答:',    generate['result'])
print('*'*10, '\nSource: ', pick_metadata_source(generate))

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "コナンはなぜ子どもの姿になったのですか？"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "コナンはなぜ子どもの姿になったのですか？",
  "context": "名探偵コナン\n\nテレビアニメをはじめ、多数のメディア化がされている青山剛昌の代表作。天才高校生探偵工藤新一は、謎の集団黒ずくめの組織に毒薬を飲まされてしまい、体が縮んで小学生の姿になってしまう。新一は、正体を隠すために江戸川コナンと名乗り、幼馴染・毛利蘭の父が営む毛利探偵事務所に身を寄せたコナンは、元の体に戻るために数々の難事件を解決しながら、黒ずくめの組織の謎に迫っていく。小学館「週刊少年サンデー」1994年5号から連載。第46回「小学館漫画賞」少年部門受賞。\n\n正式名称\n名探偵コナン\n\nふりがな\nめいたんていこなん\n\n作者\n青山 剛昌\n\nジャンル\n推理・ミステリー\n\nレーベル\n少年サンデーコミックス(小学館)\n\n巻数\n既刊103巻\n\nあらすじ\n高校生探偵として有名な工藤新一は、ある日黒ずくめの組織による怪しげな取引を目撃したため、口封じとして毒薬を飲まされてしまう。新一は一命を取り留めるも、体が縮んで小学生の姿になってしまった。自分の生存が知られたら周囲の人間にも危険が及ぶと考えた新一は、眼鏡で変装し江戸川コナンと偽名を使って正体を隠す。そして黒ずくめの組織の情報を集める為に、幼なじみの毛利蘭の父親、毛利小五郎が営む毛利探偵事務所に転がり込んだ。コナンはより多くの情報を集めるために毛利探偵事務所に寄越された依頼や、行く先で出くわした事件などを小五郎を身代わりにして解決し

In [ ]:
query = '金田一とはどのような人物ですか？'
generate = original_llm(query)
print('回答:', generate)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "金田一とはどのような人物ですか？"
  ]
}
金田一は、日本の推理小説家横溝正史の作品に登場する架空の探偵です。彼は名探偵として知られており、鋭い観察力と推理力を持ち、難解な事件を解決する能力を持っています。金髪で長身の容姿であり、頭脳明晰で冷静沈着な性格です。彼は多くの人々から信頼され、様々な難事件を解決してきました。金田一シリーズは、日本の推理小説の代表的な作品として知られています。[llm/end] [1:llm:OpenAIChat] [15.15s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "金田一は、日本の推理小説家横溝正史の作品に登場する架空の探偵です。彼は名探偵として知られており、鋭い観察力と推理力を持ち、難解な事件を解決する能力を持っています。金髪で長身の容姿であり、頭脳明晰で冷静沈着な性格です。彼は多くの人々から信頼され、様々な難事件を解決してきました。金田一シリーズは、日本の推理小説の代表的な作品として知られています。",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
回答: 金田一は、日本の推理小説家横溝正史の作品に登場する架空の探偵です。彼は名探偵として知られており、鋭い観察力と推理力を持ち、難解な事件を解決する能力を持っています。金髪で長身の容姿であり、頭脳明晰で冷静沈着な性格です。彼は多くの人々から信頼され、様々な難事件を解決してきました。金田一シリーズは、日本の推理小説の代表的な作品として知られています。


In [ ]:
query = '金田一とはどのような人物ですか？'
generate = qa_chain(query)
print('*'*10, '\n回答:',    generate['result'])
print('*'*10, '\nSource: ', pick_metadata_source(generate))

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "金田一とはどのような人物ですか？"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "金田一とはどのような人物ですか？",
  "context": "⾦⽥⼀少年の事件簿\n無料で読む・試し読み\n後に⻑期シリーズ化される『⾦⽥⼀少年の事件簿』シリーズ1作⽬。さとうふみやが作画、case2巻\nまでは⾦成陽三郎が原作、それ以降は天樹征丸（樹林伸）が原作・原案を務める。⾼校⽣の⾦⽥⼀\n⼀が遭遇する難事件を、幼馴染の七瀬美雪や、剣持勇警部、明智健悟警視などの協⼒の元、名探偵\nと呼ばれた祖⽗譲りの推理⼒で解いていくミステリ漫画。作品は、犯⼈当てに読者参加が可能な、\nフェアなミステリを意識して作られており、事件の解決編前には、ミステリ⽤語で⾔う所のQ .E.D.\nポイントとして、「謎は解けた!」などのハジメの台詞が⼊り、証拠が出揃った事が明⾔される。作\n品を注意深く読めば、その時点まで描かれた事から、事件の真相を、読者⾃⾝が推理できる仕掛け\nとなっている。⻑編では、クローズド・サークル状況での連続殺⼈物である事が多い。証拠と論理\nによる犯⼈の絞り込みが、物語のクライマックスであるが、その過程、動機、⽅法、関係者間のミ\nッシング・リンクなどの⽅に話の焦点が置かれる事もある。犯⼈には、「怪⼈名」という⼆つ名が\n\nさとうふみやの『⾦⽥⼀少年の事件簿』のスピンオフ作品。原作で事件を解決したあと、⼀泊⼆⽇\n旅⾏に出掛けた先で、⾦⽥⼀⼀と七瀬美雪が新たな騒動に巻き込まれる姿を⼀話完結で描いたコメ\nディ作品。「マンガボック... 関連ページ︓ ⾦⽥⼀少年の⼀泊⼆⽇⼩旅⾏\n⾦⽥⼀少

# end

---